In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torchvision
import torchvision.transforms
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
from torch.utils.data import DataLoader

In [ ]:
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_data = torchvision.datasets.ImageFolder(
                    root='./data/trainset/',
                    transform=transforms
                    )

test_data = torchvision.datasets.ImageFolder(
                    root= './data/test/', 
                    transform=transforms)

train_set_ratio = 0.8
size = [int(len(train_data)*train_set_ratio), len(train_data) - int(len(train_data)*train_set_ratio)]

train_dataset, valid_dataset = torch.utils.data.random_split(dataset=train_data, lengths=size)

train_loader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=64,
                    num_workers=2,
                    shuffle=True
                    )

valid_loader = torch.utils.data.DataLoader(
                    valid_dataset,
                    batch_size=64,
                    num_workers=2,
                    shuffle=True
                    )

test_loader = torch.utils.data.DataLoader(
                    test_data,
                    batch_size=64,
                    num_workers=2,
                    shuffle=False
                    )
print(train_data)
print(test_data)

In [ ]:
class Classifier(nn.Module):
    """Convnet Classifier"""
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 2
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 3
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 4
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 5
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 6
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )
        # Logistic Regression
        self.clf = nn.Linear(256, 2)

    def forward(self, x):
        return self.clf(self.conv(x).squeeze())

In [ ]:
cuda_available = torch.cuda.is_available()
print(cuda_available)

## Hyper-parameters of the model

In [40]:
learning_rate = 1e-3

In [41]:
clf = Classifier()
if cuda_available:
    clf = clf.cuda()
optimizer = torch.optim.Adam(clf.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [72]:
epoches= 50
for epoch in range(epoches):
    losses = []
    # Train
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        if cuda_available:
            inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = clf(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.item())
        
        if batch_idx%50==0:
            print('Epoch : %d Loss : %.3f ' % (epoch, np.mean(losses)))
    
    # Evaluate
    clf.eval()
    total = 0
    correct = 0
    for batch_idx, (inputs, targets) in enumerate(valid_loader):
        if cuda_available:
            inputs, targets = inputs.cuda(), targets.cuda()

        outputs = clf(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    print('Epoch : %d Test Acc : %.3f' % (epoch, 100.*correct/total))
    print('--------------------------------------------------------------')
    clf.train()

Epoch : 0 Loss : 0.005 
Epoch : 0 Loss : 0.019 
Epoch : 0 Loss : 0.018 
Epoch : 0 Loss : 0.019 
Epoch : 0 Loss : 0.022 
Epoch : 0 Test Acc : 82.000
--------------------------------------------------------------
Epoch : 1 Loss : 0.002 
Epoch : 1 Loss : 0.010 
Epoch : 1 Loss : 0.011 
Epoch : 1 Loss : 0.010 
Epoch : 1 Loss : 0.011 
Epoch : 1 Test Acc : 81.000
--------------------------------------------------------------
Epoch : 2 Loss : 0.002 
Epoch : 2 Loss : 0.007 
Epoch : 2 Loss : 0.009 
Epoch : 2 Loss : 0.010 
Epoch : 2 Loss : 0.012 
Epoch : 2 Test Acc : 80.000
--------------------------------------------------------------
Epoch : 3 Loss : 0.002 
Epoch : 3 Loss : 0.017 
Epoch : 3 Loss : 0.015 
Epoch : 3 Loss : 0.015 
Epoch : 3 Loss : 0.014 
Epoch : 3 Test Acc : 81.000
--------------------------------------------------------------
Epoch : 4 Loss : 0.001 
Epoch : 4 Loss : 0.009 
Epoch : 4 Loss : 0.008 
Epoch : 4 Loss : 0.009 
Epoch : 4 Loss : 0.012 
Epoch : 4 Test Acc : 80.000
--------

KeyboardInterrupt: 

## Evaluating of testset 

In [148]:
results = [[]]
for batch_idx, (inputs, targets) in enumerate(test_loader):
    if cuda_available:
        inputs, targets = inputs.cuda(), targets.cuda()
    outputs = clf(inputs)
    _, predicted = torch.max(outputs.data, 1)
    results = np.append(results, predicted.cpu().numpy())

results = np.int8(results)

In [147]:
df = pd.DataFrame({ 'id': range(1, len(results)+1),
                    'label': results})
df['label'].replace([0,1], ['Cat','Dog'], inplace=True)
df[df.columns].to_csv('submisstion.csv',index=False)